Manuel Rubio y Rubén Castillo 

Librerias

In [103]:
import pandas as pd
import numpy as np
from collections import Counter

Funciones

In [104]:
def entropia(x):
    # x es el número que hay de cada clase
    suma=0
    cosas=1/sum(x)
    x = [elemento * cosas for elemento in x]
    for i in x:
        suma+= -np.log2(i)*i
    return suma

def entropia_gini(x):
    # x es el número que hay de cada clase
    suma=0
    cosas=1/sum(x)
    x = [elemento * cosas for elemento in x]
    for i in x:
        suma+= -np.log2(i)*(1-i)
    return suma

def ganar_información(df,i,metodo=entropia):
    valores_distintos=set(df[i])
    total=len(df[i])
    valor=0
    for j in valores_distintos:
        indices= list(df[i]==j)
        total_subconjunto=sum(indices)
        frecuencias = Counter(df.loc[indices,'PlayTennis'])
        x=list(frecuencias.values())
        
        
        valor+= total_subconjunto/total*metodo(x)

    frecuencias = Counter(df['PlayTennis'])
    x=list(frecuencias.values())
    entropy=metodo(x)
    valor=entropy-valor
    return(valor)

# Datos:

In [105]:
data = {'Outlook':    ['s','s','o','r','r','r','o','s','s','r','s','o','o','r'],
        'Temperature':['h','h','h','m','c','c','c','m','c','m','m','m','h','m'],
        'Humidity':   ['h','h','h','h','n','n','n','h','n','n','n','h','n','h'],
        'Windy':      [False,True,False,False,False,True,True,False,False,False,True,True,False,True],
        'PlayTennis':[0,0,1,1,1,0,1,0,1,1,1,1,1,0]}

df = pd.DataFrame(data)

print(df)

frecuencias = Counter(data["PlayTennis"])
a=frecuencias[0]
b=frecuencias[1]
print(f"\nTengo a {a} que no juegan al tenis y a {b} que si juegan")

   Outlook Temperature Humidity  Windy  PlayTennis
0        s           h        h  False           0
1        s           h        h   True           0
2        o           h        h  False           1
3        r           m        h  False           1
4        r           c        n  False           1
5        r           c        n   True           0
6        o           c        n   True           1
7        s           m        h  False           0
8        s           c        n  False           1
9        r           m        n  False           1
10       s           m        n   True           1
11       o           m        h   True           1
12       o           h        n  False           1
13       r           m        h   True           0

Tengo a 5 que no juegan al tenis y a 9 que si juegan


### Entropía de los datos:

In [106]:
x=list(frecuencias.values())
entropia(x)
clases=[]

# Cálculo de la primera característica

## Cálculo de la mejor variable por el método de la entropía

In [107]:
variables=list(df.columns)
maximo=0
for i in variables[:-1]:
    ganancia=ganar_información(df,i)
    if ganancia>maximo:
        maximo=ganancia
        variable=i
    print(f"La variable {i} da una ganancia de información de {ganancia}")
print(f"\nLa mejor variable es {variable} con una ganancia de información de {maximo}")

La variable Outlook da una ganancia de información de 0.24674981977443933
La variable Temperature da una ganancia de información de 0.02922256565895487
La variable Humidity da una ganancia de información de 0.15183550136234159
La variable Windy da una ganancia de información de 0.04812703040826949

La mejor variable es Outlook con una ganancia de información de 0.24674981977443933


Me quedo con la variable Outlook y hago los 3 nodos que genera:

In [108]:
caracteristicas=set(df["Outlook"])
for i in caracteristicas:
    frecuencias = Counter(df[df["Outlook"]==i]["PlayTennis"])
    print(f'La caracteristica {i} genera un nodo {frecuencias}')


La caracteristica r genera un nodo Counter({1: 3, 0: 2})
La caracteristica s genera un nodo Counter({0: 3, 1: 2})
La caracteristica o genera un nodo Counter({1: 4})


Con esto generamos 2 nodos nuevos, uno con r y otro con s, mientras que sale una hoja con o al ser un grupo homogéneo

## Con el método de gini:

In [109]:
maximo=0
for i in variables[:-1]:
    ganancia=ganar_información(df,i, metodo=entropia_gini)
    if ganancia>maximo:
        maximo=ganancia
        variable=i
    print(f"La variable {i} da una ganancia de información de {ganancia}")
print(f"\nLa mejor variable es {variable} con una ganancia de información de {maximo}")

La variable Outlook da una ganancia de información de 0.4054685786871174
La variable Temperature da una ganancia de información de -0.09777296114265299
La variable Humidity da una ganancia de información de -0.5587260969708601
La variable Windy da una ganancia de información de -0.16243456792493194

La mejor variable es Outlook con una ganancia de información de 0.4054685786871174


Sale la misma.

# Cálculo segunda característica

## Para la primera rama (rainy):

In [110]:
subset_rainy=df[df["Outlook"]=="r"]
subset_rainy=subset_rainy.drop(columns=["Outlook"])
print(subset_rainy)

   Temperature Humidity  Windy  PlayTennis
3            m        h  False           1
4            c        n  False           1
5            c        n   True           0
9            m        n  False           1
13           m        h   True           0


In [111]:
variables=list(subset_rainy.columns)[:-1]
maximo=0
for i in variables:
    ganancia=ganar_información(subset_rainy,i)
    if ganancia>maximo:
        maximo=ganancia
        variable=i
    print(f"La variable {i} da una ganancia de información de {ganancia}")
print(f"\nLa mejor variable es {variable} con una ganancia de información de {maximo}")

La variable Temperature da una ganancia de información de 0.01997309402197489
La variable Humidity da una ganancia de información de 0.01997309402197489
La variable Windy da una ganancia de información de 0.9709505944546686

La mejor variable es Windy con una ganancia de información de 0.9709505944546686


Seleccionamos la variable windy

In [112]:
caracteristicas=set(subset_rainy["Windy"])
for i in caracteristicas:
    frecuencias = Counter(subset_rainy[subset_rainy["Windy"]==i]["PlayTennis"])
    print(f'La caracteristica {i} genera un nodo {frecuencias}')


La caracteristica False genera un nodo Counter({1: 3})
La caracteristica True genera un nodo Counter({0: 2})


Aquí ya vemos que se nos generan 2 hojas al separarnos en grupos homogéneos los datos

## Para la segunda rama (sunny)

In [113]:
subset_sunny=df[df["Outlook"]=="s"]
subset_sunny=subset_sunny.drop(columns=["Outlook"])
print(subset_sunny)

   Temperature Humidity  Windy  PlayTennis
0            h        h  False           0
1            h        h   True           0
7            m        h  False           0
8            c        n  False           1
10           m        n   True           1


In [114]:
variables=list(subset_sunny.columns)[:-1]
maximo=0
for i in variables:
    ganancia=ganar_información(subset_sunny,i)
    if ganancia>maximo:
        maximo=ganancia
        variable=i
    print(f"La variable {i} da una ganancia de información de {ganancia}")
print(f"\nLa mejor variable es {variable} con una ganancia de información de {maximo}")

La variable Temperature da una ganancia de información de 0.5709505944546686
La variable Humidity da una ganancia de información de 0.9709505944546686
La variable Windy da una ganancia de información de 0.01997309402197489

La mejor variable es Humidity con una ganancia de información de 0.9709505944546686


Seleccionamos la variable Humidity

In [115]:
caracteristicas=set(subset_sunny["Humidity"])
for i in caracteristicas:
    frecuencias = Counter(subset_sunny[subset_sunny["Humidity"]==i]["PlayTennis"])
    print(f'La caracteristica {i} genera un nodo {frecuencias}')


La caracteristica h genera un nodo Counter({0: 3})
La caracteristica n genera un nodo Counter({1: 2})
